In [57]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests, re
import itertools as it
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import warnings
warnings.filterwarnings('ignore')

In [58]:
## Get the Url list
def get_url(category):
    category = input('Enter the Category:')
    base_url = 'https://careers.tencent.com/search.html?query=co_1,ot_40001001&sc=1'
    search = base_url.replace('ot_40001001',category)
    driver = webdriver.Chrome()
    driver.get(search)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    maxpage = driver.find_elements_by_class_name('page-text')[-1].text
    driver.quit()
    #maxpage = soup.find_all('span', class_='page-text')[-1].get_text()
    Page = [str(i+1) for i in range(int(maxpage))]
    seed = []
    url = search[:-5]+'&index='
    for num in Page:
        seed_urls = url+num+'&sc=1'
        seed.append(seed_urls)
    return seed

In [59]:
#技术研发 = 'ot_40001001'
#质量管理 = 'ot_40001002'
#技术运营 = 'ot_40001003'
#安全技术 = 'ot_40001004'
#AI算法与大数据 = 'ot_40001005'
#企管 = 'ot_40001006'

# 中国 = co_1

# 深圳 = c1_1

In [87]:
### Scraping

df = pd.DataFrame()
for i in get_url(category):
    browser = webdriver.Chrome()
    browser.get(i)
    element = browser.find_elements_by_class_name("recruit-list-link")
    contents = []
    for j in element:
        j.click()
        browser.switch_to_window(browser.window_handles[-1])
        ## Link
        link = browser.current_url
        soup = BeautifulSoup(browser.page_source,'html.parser')

        ## Job Title
        title = soup.find_all('div',class_='work-title')[0].get_text()
        ## Reponsibility
        jd = soup.find_all('li', class_='explain-item')[0].get_text()
        ## Requirement
        quali = soup.find_all('li', class_='explain-item')[1].get_text()

        
        tips = soup.find_all('p',class_="recruit-tips")[0].get_text()
        ## 事业群
        group = tips.split(" | ")[0]
        ## 城市
        city = tips.split(" | ")[1]

        ## 时间
        if '|' in string.split(" | ")[-1]:
            time = string.split(" | ")[-1][2:]
        else:
            time = string.split(" | ")[-1].strip()
        
        browser.switch_to_window(browser.window_handles[0])
        contents.append([link,title,jd,quali,group,city,time])
        df_row = pd.DataFrame(contents)
    browser.quit()
    df = df.append(df_row)

Enter the Category:ot_40001002


In [82]:
df.columns = ['Link','Title','JD','Requirements','Group','City','Time']
df
df.to_csv('腾讯质量管理.csv', encoding='utf_8_sig')

In [88]:
df

,0,1,2,3,4,5,6
0,https://careers.tencent.com/jobdesc.html?postI...,26563-系统测试,负责PC或移动端游戏平台相关需求的澄清和测试，制定相应的测试策略和测试计划，达成项目质量目标...,计算机相关专业本科以上学历，2年以上软件测试或开发经验；热爱软件测试，拥有出色的测试用例设计...,IEG,深圳,2020年08月18日
1,https://careers.tencent.com/jobdesc.html?postI...,25927-安全专项测试工程师（成都）,负责移动游戏和应用APP的风险分析与漏洞挖掘；负责研发漏洞挖掘工具和方法，优化测试方案和效率。,本科以上学历，2年以上开发或测试经验；责任心强，良好的沟通能力和团队协作能力；至少熟悉一种编...,IEG,成都,2020年08月18日
2,https://careers.tencent.com/jobdesc.html?postI...,25927-游戏测试经理,负责项目目标规划、需求分解、计划排期和总结汇报；根据需求评估并制定搭建开发框架，承担开发设计...,本科及以上学历，有2年以上软件开发经验，有测试和开发相关工作经验优先；精通C++、C、或py...,IEG,成都,2020年08月18日
3,https://careers.tencent.com/jobdesc.html?postI...,PCG04-研发效能改进工程师,1、负责产品团队内研发效能改进方案设计，制定整体的解决方案，并推动在业务团队内的落地实施； ...,1、国家统招本科及以上学历，计算机、软件工程、数学等相关专业；2、杰出的自我驱动学习能力，能...,PCG,深圳,2020年08月18日
4,https://careers.tencent.com/jobdesc.html?postI...,25926-游戏安全测试工程师（深圳）,负责移动游戏安全组件版本规划、版本质量管理，对最终版本质量负责负责移动游戏安全版本的安全风险...,大学本科以上学历，计算机专业优先3年以上测试相关领域行业工作经验（游戏测试、游戏运营等相关经...,IEG,深圳,2020年08月18日
...,...,...,...,...,...,...,...
8,https://careers.tencent.com/jobdesc.html?postI...,TME-腾讯音乐Web质量测试工程师（深圳）,主要负责腾讯音乐互联网产品WEB端、对应后端相关的测试分析、执行。包括功能测试、模块测试、系...,本科及以上学历，计算机相关专业，3年以上web端开发或测试经验；熟悉Javasript/HT...,TME,深圳,2020年08月18日
9,https://careers.tencent.com/jobdesc.html?postI...,TME-视频专项测试,负责直播，视频相关的需求的质量（效果和性能）保证工作；分析竞品并制定测试方案，对清晰度，美颜...,本科以上学历，计算机相关专业；3年以上相关工作经验；熟悉音视频编解码、视频图像处理，直播相关...,TME,深圳,2020年08月18日
0,https://careers.tencent.com/jobdesc.html?postI...,TME-腾讯音乐质量管理工程师,基于流程框架推动并优化运营质量管理，利用过程审计、度量分析，识别研发和运营过程管理改进点，并...,本科或以上学历，计算机或相关理工科专业，3年以上工作经验，了解或熟悉移动互联网行业运营质量管...,TME,深圳,2020年08月18日
1,https://careers.tencent.com/jobdesc.html?postI...,TME-腾讯音乐全民K歌测试工程师（深圳）,本科及以上学历，计算机软件相关专业，熟悉Android/iOS系统；1年及以上软件测试经验，...,主要负责全民K歌前后端质量保障工作，深度挖掘及分析外网问题，制定测试方案，参与产品需求、设计...,TME,深圳,2020年08月18日
